## Question Paper Generator

In [1]:
import PyPDF2
import pandas as pd
from fpdf import FPDF
import requests
import random
import json
import re

In [2]:
OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL = "llama3"

def ask_ollama(prompt):
    payload = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": "You are a precise JSON generator."},
            {"role": "user", "content": prompt}
        ],
        "stream": False
    }
    response = requests.post(OLLAMA_URL, json=payload)
    response.raise_for_status()
    return response.json()["message"]["content"]


In [5]:
def extract_json_from_text(text):
    match = re.search(r'(\[.*\])', text, re.DOTALL)
    if not match:
        raise ValueError("No JSON array found")
    return json.loads(match.group(1))

In [7]:
file_path = input("Enter question text filename (example: questions.txt): ")

with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Loaded text characters:", len(raw_text))


Enter question text filename (example: questions.txt):  Sample_Question_Paper.txt


Loaded text characters: 1173


In [29]:
prompt = f"""
You extract exam questions.

STRICT RULES:
- Output ONLY valid JSON
- No explanations
- No markdown
- No extra text

EXAMPLE:

INPUT:
MCQ: What is 2 + 2? Options: A) 3 B) 4 C) 5 D) 6
Short Answer: Define gravity.

OUTPUT:
[
  {{
    "question_text": "What is 2 + 2? Options: A) 3 B) 4 C) 5 D) 6",
    "type": "MCQ"
  }},
  {{
    "question_text": "Define gravity.",
    "type": "Short Answer"
  }}
]

NOW PROCESS THIS INPUT:
{raw_text}
"""

In [31]:
response = ask_ollama(prompt)

try:
    questions = extract_json_from_text(response)
except Exception as e:
    print("❌ JSON extraction failed")
    print(response)
    raise e

print("✅ Questions extracted:", len(questions))


✅ Questions extracted: 15


In [32]:
total_marks = int(input("Total marks (25 / 50 / 100): "))
mcq_count = int(input("MCQs (0 = auto): "))
short_count = int(input("Short answers (0 = auto): "))
long_count = int(input("Long answers (0 = auto): "))

MARKS = {"MCQ": 1, "Short Answer": 2, "Long Answer": 5}


Total marks (25 / 50 / 100):  25
MCQs (0 = auto):  0
Short answers (0 = auto):  0
Long answers (0 = auto):  0


In [43]:
# Shuffle questions for randomness
random.shuffle(questions)

# Separate questions by type
mcqs = [q for q in questions if q["type"] == "MCQ"]
shorts = [q for q in questions if q["type"] == "Short Answer"]
longs = [q for q in questions if q["type"] == "Long Answer"]

MARKS = {
    "MCQ": 1,
    "Short Answer": 2,
    "Long Answer": 5
}

# AUTO MODE: teacher entered 0 0 0
if mcq_count == 0 and short_count == 0 and long_count == 0:
    remaining_marks = total_marks

    mcq_count = min(len(mcqs), remaining_marks // MARKS["MCQ"])
    remaining_marks -= mcq_count * MARKS["MCQ"]

    short_count = min(len(shorts), remaining_marks // MARKS["Short Answer"])
    remaining_marks -= short_count * MARKS["Short Answer"]

    long_count = min(len(longs), remaining_marks // MARKS["Long Answer"])

# SAFETY CHECKS
mcq_count = min(mcq_count, len(mcqs))
short_count = min(short_count, len(shorts))
long_count = min(long_count, len(longs))

print("\n✅ Final Question Distribution:")
print(f"MCQs        : {mcq_count}")
print(f"Short Answer: {short_count}")
print(f"Long Answer : {long_count}")



✅ Final Question Distribution:
MCQs        : 5
Short Answer: 5
Long Answer : 2


In [45]:
paper = []
paper.append("QUESTION PAPER")
paper.append(f"Total Marks: {total_marks}")
paper.append("\nSection A: MCQs")

for i, q in enumerate(mcqs[:mcq_count], 1):
    paper.append(f"{i}. {q['question_text']} ({MARKS['MCQ']} mark)")

paper.append("\nSection B: Short Answers")
for i, q in enumerate(shorts[:short_count], 1):
    paper.append(f"{i}. {q['question_text']} ({MARKS['Short Answer']} marks)")

paper.append("\nSection C: Long Answers")
for i, q in enumerate(longs[:long_count], 1):
    paper.append(f"{i}. {q['question_text']} ({MARKS['Long Answer']} marks)")

In [47]:
paper

['QUESTION PAPER',
 'Total Marks: 25',
 '\nSection A: MCQs',
 '1. What is the capital of France? Options: A) Paris B) London C) Berlin D) Madrid (1 mark)',
 '2. Which element has atomic number 1? Options: A) Helium B) Hydrogen C) Oxygen D) Carbon (1 mark)',
 '3. What is 5 + 7? Options: A) 10 B) 11 C) 12 D) 13 (1 mark)',
 '4. Which planet is known as the Red Planet? Options: A) Earth B) Mars C) Jupiter D) Venus (1 mark)',
 "5. Who wrote 'Romeo and Juliet'? Options: A) Shakespeare B) Dickens C) Orwell D) Twain (1 mark)",
 '\nSection B: Short Answers',
 '1. Define photosynthesis in one sentence. (2 marks)',
 '2. Explain supervised learning in 2-3 sentences. (2 marks)',
 '3. What is the difference between mean and median? (2 marks)',
 '4. Explain the difference between AC and DC current. (2 marks)',
 '5. List two main causes of climate change. (2 marks)',
 '\nSection C: Long Answers',
 '1. Describe the water cycle and its importance to the environment. (5 marks)',
 '2. Discuss the advantag

In [49]:
#with open("Question_Paper.txt", "w", encoding="utf-8") as f:
#    f.write("\n".join(paper))

#print("✅ Question_Paper.txt generated successfully")